In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import xarray as xr
from datetime import datetime
import cv2
import tabular.subset_by_images
import matplotlib.pyplot as plt
import cv_experiments.cv_common as cc
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
image_to_boat = lambda fname: os.path.basename(fname).split('_')[0].split('-')[1]
image_to_datetime = lambda fname: np.datetime64(tabular.subset_by_images.image_to_datetime(fname))
image_to_camera = lambda fname: os.path.splitext(os.path.basename(fname))[0].split('_')[2].split('-')[1]

In [ ]:
def pr2line(pr, shape):
    pitch_slope,roll_slope = pr
    height,width = shape[:2]
    y0 = height*(0.5-pitch_slope)
    yoffset = -roll_slope*width/2
    return (0, int(y0-yoffset)), (width, int(y0+yoffset))

def show_images(images):
    for row in images:
        lat, lon = None, None
        if "dataframe" in row:
            lat, lon = row["dataframe"][["latitude", "longitude"]].mean()
        print(f"{row['boat']} at {pd.to_datetime(row['datetime']):%Y-%m-%d %H:%M:%S}, {f'lat={lat:.3f}, lon={lon:.3f}, ' if lat and lon else ''}camera {row['camera']}:")
        if "msg" in row: print(row["msg"])
        img = cv2.cvtColor(row["mat"], cv2.COLOR_GRAY2RGB)
        if "horizon_inferred" in row:
            c1,c2 = pr2line(row["horizon_inferred"], img.shape)
            cv2.line(img, c1, c2, (0, 0, 255), 3, cv2.LINE_AA)
        if "horizon_manual" in row:
            c1,c2 = pr2line(row["horizon_manual"], img.shape)
            cv2.line(img, c1, c2, (255, 0, 0), 3, cv2.LINE_AA)
        plt.imshow(img, vmin=0, vmax=255)
        plt.show()

In [ ]:
image_paths = sorted(f.path for f in os.scandir("../representatives/correlation/") if f.is_file() and os.path.basename(f.path)[0] != '.')
images = [{"boat": image_to_boat(path), "datetime": image_to_datetime(path), "camera": image_to_camera(path), "mat": cv2.imread(path, 0)} for path in image_paths]
show_images(images)

In [ ]:
for row in images: row["mat"] = cc.undistort(row["mat"], "large" if row["mat"].shape[0] > 1000 else "small")
show_images(images)

In [ ]:
# Tuned by hand; will break if the input file list is changed
manual_horizons = [(-0.110, -0.078), (0.100, 0.088), (0.010, -0.135), (0.127, 0.269)]
for row, horiz in zip(images, manual_horizons): row["horizon_manual"] = horiz
show_images(images)

In [ ]:
excerpt_to_boat = lambda fname: os.path.splitext(os.path.basename(fname))[0].split('_')[1]
excerpt_paths = sorted(f.path for f in os.scandir("../tabular/excerpts") if f.is_file() and os.path.basename(f.path)[0] != '.')
excerpts = {excerpt_to_boat(fname): xr.load_dataset(fname) for fname in excerpt_paths}

In [ ]:
time_threshold = np.timedelta64(10, "s")
for row in images:
    row["tabular"] = excerpts[row["boat"]].sel(time=slice(row["datetime"]-time_threshold, row["datetime"]+time_threshold))
print(list(excerpts[images[0]["boat"]].variables))

In [ ]:
relevant_variables = ["ROLL", "PITCH", "HDG", "ROLL_WING", "PITCH_WING", "HDG_WING", "WING_ANGLE", "latitude", "longitude"]
for row in images:
    row["dataframe"] = row["tabular"].to_dataframe()[relevant_variables]

In [ ]:
def pr_offset_heading(pitch, roll, heading_offset):  # Takes and gives all values in degrees!
    atan, cos, sin, tan, deg, rad = np.arctan, np.cos, np.sin, np.tan, np.degrees, np.radians
    new_pitch = deg(atan(cos(rad(heading_offset))*tan(rad(pitch)) - sin(rad(heading_offset))*tan(rad(roll))))
    new_roll  = deg(atan(cos(rad(heading_offset))*tan(rad(roll))  + sin(rad(heading_offset))*tan(rad(pitch))))
    return new_pitch, new_roll

def pr_offset_camera(pitch, roll, camera):
    if camera == "2": return pr_offset_heading(pitch, roll, 90)
    if camera == "3": return pr_offset_heading(pitch, roll, -90)

In [ ]:
fov = {("h", "small"): 98,
       ("v", "small"): 54.5,
       ("h", "large"): 103,
       ("v", "large"): 57.5}

threshold_before = np.timedelta64(int(0.01*1000), "ms")
threshold_after = np.timedelta64(int(0.01*1000), "ms")
for row in images:
    little_slice = slice(row["datetime"]-threshold_before, row["datetime"]+threshold_after)
    craft_pitch, craft_roll = row['dataframe'][['PITCH', 'ROLL']][little_slice].mean()
    wing_pitch, wing_roll = row['dataframe'][['PITCH_WING', 'ROLL_WING']][little_slice].mean()
    image_pitch, image_roll = pr_offset_camera(wing_pitch, wing_roll, row["camera"])
    pitch_ratio = -image_pitch/fov[("v", "large" if row["mat"].shape[0] > 1000 else "small")]
    row["horizon_inferred"] = (pitch_ratio, 0)
    row["msg"] = f"\
        Craft pitch: {craft_pitch:.2f}\n\
        Craft roll: {craft_roll:.2f}\n\
        Wing pitch: {wing_pitch:.2f}\n\
        Wing roll: {wing_roll:.2f}\n\
        Image pitch: {image_pitch:.2f}\n\
        Image roll: {image_roll:.2f}\n\
        Pitch ratio: {pitch_ratio:.2f}\n\
        "
show_images(images)